In [1]:
pip install langchain

In [5]:
pip install langchain_community openai

In [3]:
!pip install openai

Haciendo los import y configurando la api key

In [78]:
import os
from openai import OpenAI as OpenAI

from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool, load_tools
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.llms import OpenAI as OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate

import json

from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

client = OpenAI(
    api_key = OPENAI_API_KEY
)
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [14]:
#ChatBot Type
chat = ChatOpenAI(model="gpt-4o-mini")
#chat_text = "Dame las entidades y relaciones para una base de datos de una veterinaria" #Aqui van las preguntas
#messages = [HumanMessage(content=chat_text)]
#print(chat.predict_messages(messages).content)

<ipython-input-14-84c2753dc516>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(model="gpt-4o-mini")


# CREAR TOOLS


In [156]:
# CHAINS
# Crear una cadena (Chain) para registrar transacciones
prompt_registrar = PromptTemplate(
    input_variables=["input"],
    template=""""
    Del input recibido, analiza con precaucion y extrae los parametros necesarios para hacer el registro:
    - Tipo (Ingreso o Gasto)
    - Monto (número)
    - Categoría (texto)
    - Descripción (texto, opcional, usa '' si no está presente)

    Texto: "{input}"

    Devuelve la respuesta en formato "(tipo) registrado con (monto) en (categoria) con (descripcion).".
    """
)

chain_registrar = LLMChain(llm=chat, prompt=prompt_registrar)

tool_registrar = Tool(
    name="registrar_transaccion",
    func=lambda x: chain_registrar.run(input=x),
    description="Sirve para registrar un ingreso o gasto con tipo (Ingreso/Gasto), monto (número), categoria (texto) y descripcion (texto) (opcional).")


# CALCULO DE FLUJO DE CAJA
prompt_flujo_caja = PromptTemplate(
    input_variables=["input"],
    template="""
    Analiza el siguiente texto y extrae los parámetros para calcular el flujo de caja:
    - Ingresos (número)
    - Gastos (número)

    Texto: "{input}"

    Calcula el saldo (ingresos - gastos) y devuelve el resultado en formato:
    "🔹 Ingresos: $(ingresos) 🔹 Gastos: $(gastos)🔹 Saldo: $(saldo)"
    """
)
chain_flujo_caja = LLMChain(llm=chat, prompt=prompt_flujo_caja)

tool_flujo_caja = Tool(
    name="calcular_flujo_caja",
    func=lambda x: chain_flujo_caja.run(input=x),
    description="Calcula el flujo de caja a partir de ingresos y gastos."
)


# CONFIGURAR PRESUPUESTO
prompt_presupuesto = PromptTemplate(
    input_variables=["input"],
    template="""
    Analiza el siguiente texto y extrae los parámetros para configurar un presupuesto:
    - Categoría (texto)
    - Monto (número)

    Texto: "{input}"

    Devuelve el resultado en formato: "✅ Presupuesto de ${monto} para {categoria} configurado."
    """
)

chain_presupuesto = LLMChain(llm=chat, prompt=prompt_presupuesto)

tool_presupuesto = Tool(
    name="configurar_presupuesto",
    func= lambda x: chain_presupuesto.run(input=x),
    description="Configura un presupuesto para una categoría."
)

# RESPONDER PREGUNTAS FINANCEIRAS
prompt_preguntas = PromptTemplate(
    input_variables=["input"],
    template="""
    Responde la siguiente pregunta financiera de manera clara y sencilla:
    "{input}"
    Usa toda la informacion disponible para responder la pregunta. Es crucial que consultes la memoria para obtener datos relevantes.
    """
)
chain_preguntas = LLMChain(llm=chat, prompt=prompt_preguntas)

tool_preguntas = Tool(
    name="responder_pregunta",
    func=lambda x: chain_preguntas.run(input=x),
    description="Responde preguntas financieras con la informacion financieras generales o de la informacion del usuario y la memoria"
)

In [175]:
# Añadir la herramienta procesar_multiples tools e indicaciones

# Prompt para descomponer el mensaje
prompt_descomponer = PromptTemplate(
    input_variables=["input"],
    template="""
    Analiza el siguiente texto y descompónlo en una lista de tareas a ejecutar. Para cada tarea, identifica:
    - Herramienta a usar: "registrar_transaccion", "calcular_flujo_caja", "configurar_presupuesto",  "responder_pregunta", etc...
    - Input específico para esa herramienta (como texto o parámetros)

    Texto: {input}

    Para el output:
    Devuelve el resultado como una lista de objetos JSON, donde cada objeto tiene:
    "tool": nombre de la herramienta
    "input": texto o parámetros para esa herramienta

    Ejemplo:
    Input: "Registra un ingreso de $100 en ventas y calcula el flujo de caja con ingresos de $500 y gastos de $300."
    Output: [
        {{"tool": "registrar_transaccion", "input": "Registra un ingreso de $100 en ventas."}},
        {{tool": "calcular_flujo_caja", "input": "Calcula el flujo de caja con ingresos de $500 y gastos de $300."}}
    ]
    """
)
#chain para descomponer
chain_descomponer = LLMChain(llm=chat, prompt=prompt_descomponer)

#Funcion para la implementacion de tool
def procesar_multiples_tool(input_text):
    tareas_json = chain_descomponer.run(input=input_text)
    try:
        tareas = json.loads(tareas_json)
    except json.JSONDecodeError as e:
        return f"Error al procesar las tareas: {str(e)}"

    herramientas = {
        "registrar_transaccion": lambda x: chain_registrar.run(input=x),
        "calcular_flujo_caja": lambda x: chain_flujo_caja.run(input=x),
        "configurar_presupuesto": lambda x: chain_presupuesto.run(input=x),
        "responder_pregunta": lambda x: chain_preguntas.run(input=x)
    }
    resultados = []
    for tarea in tareas:
        tool_name = tarea["tool"]
        tool_input = tarea["input"]
        resultado = herramientas[tool_name](tool_input)
        resultados.append(resultado)
    return "\n".join(resultados)
#instancia de tool
tool_multiples = Tool(
    name="procesar_multiples",
    func=procesar_multiples_tool,
    description="Procesa un mensaje con múltiples instrucciones y ejecuta las herramientas en orden."
)

# CONFIGURAR AL AGENTE

In [132]:
memory = ConversationBufferMemory(memory_key="chat_history")

# Inicializar el agente
# Lista de herramientas
tools = [tool_registrar,tool_flujo_caja, tool_presupuesto, tool_preguntas, tool_multiples]

# Inicializar el agente
agente = initialize_agent(
    tools = tools,
    llm = chat,
    agent="conversational-react-description",
    verbose=True,
    max_iterations = 3,
    memory = memory
    )

TESTEO DEL AGENTE

In [176]:
print(agente.run("Registra un gasto de $150 en insumos y un ingreso de $200 en ventas."))
#print(agente.run("¿Qué es el flujo de caja?)
#print(agente.run("Calcula el flujo de caja con ingresos de $500 y gastos de $300."))
#print(agente.run("Registra un gasto de $200 en publiicidad."))
#print(agente.run("Cuanto he gastado en total?"))
#print(agente.run("¿Qué es el flujo de caja?"))




> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: procesar_multiples
Action Input: registrar_transaccion con tipo Gasto, monto 150, categoria "insumos", descripcion "" y registrar_transaccion con tipo Ingreso, monto 200, categoria "ventas", descripcion ""

JSONDecodeError: Expecting value: line 1 column 1 (char 0)